Langchain 프롬프트 템플릿
Langchain 프롬프트 템플릿 하는 이유 : 

```
LLM 프롬프트를 동적으로 구성하고 재사용할 수 있도록 해주는 도구

        단일 입력 : 하나의 변수로 구성된 프롬프트 템플릿
        다중 입력 : 둘 이상의 변수를 사용하는 템플릿
        ChatPromptTemplate 역할 기반 프롬프트 : 시스템/사용자 역할별 프롬프트 구성 .from_message()
        PartialPromptTemplate 활용 : 프롬프트 일부를 미리 고정하고 부분포맷티을 사용(ex. 시스템 메시지는 고정)
        프롬프트 출력 및 체인 실행 : LCEL
        프롬프트 작성 팁 : 주의사항 및 모범사례
```


In [ ]:
%pip install langchain-openai langchain-community python-dotenv openai

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# 단일 프롬프트 사용
from langchain.prompts import PromptTemplate
# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터 입니다. \n"
    "아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요.\n\n "
    "제품명 : {product_name}"
)
# PromptTemplate 객체 생성
product_prompt = PromptTemplate.from_template(template_str)
# 프롬프트 템플릿에 값 채우기
formatted_prompt = product_prompt.format(product_name="이클립스")
# 프롬프트 출력    
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터 입니다. 
아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요.

 제품명 : 이클립스


## LLM과 연결

In [9]:
# 프롬프트 | LLM 연결 -> invoke
from langchain_openai import ChatOpenAI
# OpenAI LLM 연결
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# Runnable 객체 생성 LCEL
chain = product_prompt | llm
result = chain.invoke({"product_name": "이클립스"})
# 결과 출력
print(result.content)

"이클립스, 당신의 일상에 빛나는 변화를! 매일의 순간을 특별하게 만들어주는 혁신적인 경험을 만나보세요. 지금 바로 이클립스와 함께하세요!"


In [10]:
# 다중 입력
# 다중 입력 템프릿 문자열 정의
multi_template_str = (
    "아래는 뉴스 기사 제목과 키워드 입니다.\n"
    "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요.\n\n"
    "제목 : {title}\n"
    "키워드 : {keyword}"
)

# 프롬프트 템플릿 작성
summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title", "keyword"])
# 프롬프트 템플릿에 값 채우기 (포맷팅을 통해 프롬프트 값 확인)
sample_title = "인공지능 기술의 발전과 미래"
sample_keyword = "머신러닝, 딥러닝, LLM, 랭체인, 산업 혁신"

formatted_summary_prompt = summary_prompt.format(
    title = sample_title,
    keyword = sample_keyword
)
print(formatted_summary_prompt)

아래는 뉴스 기사 제목과 키워드 입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요.

제목 : 인공지능 기술의 발전과 미래
키워드 : 머신러닝, 딥러닝, LLM, 랭체인, 산업 혁신


In [13]:
# 문자열 출력 파서 Runnable 객체  - invoke
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser() # 문자열 출력 파서 생성

# LCEL 출력
summary_chain = summary_prompt | llm | parser
# invoke 메서드로 프롬프트 실행 
summary_result = summary_chain.invoke({
    "title": sample_title,
    "keyword": sample_keyword
})
# 결과 출력
summary_result

'인공지능 기술의 발전은 머신러닝과 딥러닝의 혁신을 통해 가속화되고 있으며, 특히 대규모 언어 모델(LLM)과 랭체인 기술이 주목받고 있습니다. 이러한 기술들은 다양한 산업 분야에서 혁신을 이끌어내고 있으며, 기업들은 이를 활용하여 효율성을 높이고 새로운 비즈니스 모델을 창출하고 있습니다. 앞으로 인공지능은 더욱 발전하여 우리의 삶과 산업 구조에 큰 변화를 가져올 것으로 기대됩니다.'

In [ ]:
# ChainPromptTemplate과 역할 기반 프롬프트
# 시스템/사용자/어시스턴트 역할(role)
# 시스템 메시지: 머델의 동작을 지시
# 사용자 메시지: 실제 사용자의 입력
# 어시스턴트 메시지: 이전 모델이 응답한 내용이 있다면 대화맥락유지를 위해 활용

In [18]:
from langchain.prompts import ChatPromptTemplate
# 역할 기반 프롬프트 템플릿 생성    
system_message = ("당신은 python 분야의 뛰어난 전문가이자 조언자 입니다."
                  "사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요.")
user_message = "{question}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", user_message)
])  

# 템플릿을 이용해서 문장을 완성해 주세요 -> LLM 쓰지않음 (chat_prompt는 Runnable 객체)
sample_question = "파이썬의 클래스에 대해서 설명해주세요"
formatted_chat_prompt = chat_prompt.format_messages(question=sample_question)
# 프롬프트 출력
print(formatted_chat_prompt)



# 파이프라인을 이용해서 LLM호출 및 출력

chat_chain = chat_prompt | llm | parser
# invoke 메서드로 프롬프트 실행
chat_result = chat_chain.invoke({
    "question": sample_question
})
# 결과 출력
print(chat_result)


[SystemMessage(content='당신은 python 분야의 뛰어난 전문가이자 조언자 입니다.사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 클래스에 대해서 설명해주세요', additional_kwargs={}, response_metadata={})]
파이썬의 클래스는 객체 지향 프로그래밍(OOP)의 기본 개념 중 하나로, 객체를 생성하기 위한 청print(blueprint)입니다. 클래스는 속성(데이터)과 메서드(함수)를 포함하여, 관련된 데이터를 하나의 단위로 묶어 관리할 수 있게 해줍니다.

### 클래스의 기본 구조

클래스를 정의하는 기본적인 문법은 다음과 같습니다:

```python
class ClassName:
    def __init__(self, parameters):
        # 생성자: 객체가 생성될 때 호출되는 메서드
        self.attribute = parameters  # 속성 초기화

    def method(self):
        # 클래스의 메서드
        print("Hello, World!")
```

### 주요 구성 요소

1. **클래스 정의**: `class ClassName:` 구문을 사용하여 클래스를 정의합니다. `ClassName`은 클래스의 이름으로, 일반적으로 대문자로 시작합니다.

2. **생성자 (`__init__` 메서드)**: 클래스의 인스턴스가 생성될 때 호출되는 특별한 메서드입니다. 주로 객체의 초기 속성을 설정하는 데 사용됩니다. `self`는 인스턴스 자신을 참조합니다.

3. **속성**: 클래스 내에서 정의된 변수로, 객체의 상태를 나타냅니다. `self.attribute`와 같이 `self`를 사용하여 속성을 정의합니다.

4. **메서드**: 클래스 내에서 정의된 함수로, 객체가 수행할 수 있는 

In [19]:
# PartialPromptTemplate : 템플릿의 일부를 부분적으로 채운 새로운 템플릿
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate
role_system_template = '당신은 {role} 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요'
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

# chatprompttemplate 생성
base_chat_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    user_prompt])

partial_chat_prompt = base_chat_prompt.partial(role="주식 투자")

In [21]:
# partial로 생성된 프롬프트에 질문만 채워 츠롬프트 구성
sample_question = "현재 2025냔 5월 시장 상황에서 삼성전자 주식 전망은?"
message = partial_chat_prompt.format_messages(question=sample_question)
print(message)
    

[SystemMessage(content='당신은 주식 투자 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025냔 5월 시장 상황에서 삼성전자 주식 전망은?', additional_kwargs={}, response_metadata={})]


In [22]:
# LCEL - invkoke
partial_chat_chain = partial_chat_prompt | llm | parser
# invoke 메서드로 프롬프트 실행
partial_chat_result = partial_chat_chain.invoke({
    "question": sample_question
})
# 결과 출력
print(partial_chat_result)

2025년 5월 현재 삼성전자 주식의 전망을 평가하기 위해서는 여러 가지 요소를 고려해야 합니다. 다음은 삼성전자의 주식 전망에 영향을 미칠 수 있는 주요 요인들입니다.

1. **반도체 시장 동향**: 삼성전자는 세계 최대의 반도체 제조업체 중 하나입니다. 반도체 수요는 AI, 클라우드 컴퓨팅, IoT(사물인터넷) 등 다양한 산업에서 증가하고 있습니다. 2025년에는 이러한 기술들이 더욱 발전할 것으로 예상되며, 반도체 수요가 증가할 가능성이 높습니다. 그러나 공급 과잉이나 가격 하락 등의 리스크도 존재하므로, 시장 상황을 면밀히 분석해야 합니다.

2. **스마트폰 및 가전제품 판매**: 삼성전자는 스마트폰과 가전제품에서도 강력한 입지를 가지고 있습니다. 5G와 같은 새로운 기술의 도입이 소비자 수요에 긍정적인 영향을 미칠 수 있으며, 특히 프리미엄 제품군에서의 성장이 기대됩니다. 그러나 경쟁사와의 치열한 경쟁도 고려해야 합니다.

3. **글로벌 경제 상황**: 글로벌 경제의 성장률, 금리, 인플레이션 등은 삼성전자의 실적에 큰 영향을 미칠 수 있습니다. 특히, 미국과 중국 간의 무역 관계나 글로벌 공급망의 변화는 삼성전자의 수익성에 영향을 줄 수 있습니다.

4. **기술 혁신 및 연구개발**: 삼성전자는 지속적인 연구개발을 통해 새로운 기술을 선보이고 있습니다. 특히, 반도체 분야에서의 기술 혁신은 회사의 경쟁력을 높이는 중요한 요소입니다. 이러한 혁신이 성공적으로 시장에 적용된다면 주가에 긍정적인 영향을 미칠 수 있습니다.

5. **주식 시장 전반의 분위기**: 주식 시장의 전반적인 분위기와 투자자 심리도 삼성전자 주가에 영향을 미칠 수 있습니다. 경제 지표나 정치적 사건 등이 주식 시장에 미치는 영향을 주의 깊게 살펴봐야 합니다.

결론적으로, 삼성전자의 주식 전망은 긍정적일 수 있지만, 여러 외부 요인과 내부 요인을 종합적으로 고려해야 합니다. 투자 결정을 내리기 전에 충분한 분석과 정보를 바탕으로 신중하게 접근하는 것이 중요합니다.
